# Network centralities

**Katz centrality**: it generalizes the eigenvector centrality by assuming that nodes have also an intrinsic centrality. The centrality is the sum of the intrinsic centrality and the centrality given by the network.

$$
z =  \frac{1-\beta}{\lambda_W} W' z + \beta \mu\,, \quad \beta \in [0,1].
$$

If $\beta = 0$, the Katz centrality coincides with the eigenvector centrality. If $\beta = 1$, then $z=\mu$.

**Bonacich centrality (or Page-rank)**: it generalizes the invariant distribution centrality by assuming that nodes have also an intrinsic centrality. The centrality is the sum of the intrinsic centrality and the centrality given by the network.
$$ 
z = (1-\beta)P' z + \beta \mu\,, \quad \beta \in [0,1].
$$

If $\beta = 0$, it is the invariant distribution centrality. If $\beta = 1$, then $z=\mu$.

# Iterative methods to compute network centralities
The smartest way to compute Bonacich centrality (or Katz centrality) is by iterative methods.

Consider the dynamics
$$
\begin{cases} 
z(t+1) = (1-\beta)P'z(t) + \beta \mu  \\
z(0) = z_0
\end{cases}
$$

The transient of the dynamics is

$$
\begin{cases}
z(1) = (1-\beta)P'z(0) + \beta \mu\\
z(2) = (1-\beta)^2 (P')^2 z(0) + (1-\beta)P' \beta \mu + \beta \mu\\
\vdots\\
z(t) = (1-\beta)^t (P')^t z(0) + \sum_{i=0}^{t-1} (1-\beta)^i (P')^i \beta \mu
\end{cases}
$$

The first term vanishes as $t \to +\infty$ because $(1-\beta)P'$ is sub-stochastic, while the second term is a geometric sum. The dynamics thus converges to the limit

$$
\lim_{t \to +\infty} z(t) = (\mathbf{I}-(1-\beta) P')^{-1} \beta \mu,
$$

which is the Bonacich centrality of the graph.

**Remark 1**: You should never use direct ways to compute centralities if iterative algorithms are available. The iterative method is more efficient than the direct one as the order of the graph grows, since it does not involve the inversion of a $N \times N$ matrix.

**Remark 2**: Note that the convergence of $z(t)$ to the Bonacich centrality holds for every initial condition $z_0$ and the limit is independent of $z_0$.

**Remark 3**: Notice that the proposed method is **distributed**, i.e., the operations at single node levels do not require a complete knowledge of the network. Each node $i$ updates its state $z_i(t+1)$ by using only local information, i.e., the i-th row of $W$ and the state $z_j(t)$ of nodes $j$ that are adjacent to $j$.

Let us implement the distributed method.

In [ ]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline

G = nx.karate_club_graph()

# Fix node positions on all pictures according to spring layout
pos = nx.spring_layout(G) 
nx.draw_networkx(G, pos)

plt.show()

# compute matrices of the graph
W = nx.adjacency_matrix(G)
W = W.toarray()
degrees = np.sum(W,axis=1)
D = np.diag(degrees)
P = np.linalg.inv(D) @ W

N = G.number_of_nodes()
beta = 0.15
mu = np.ones((N,1))/N

In [ ]:
# arbitrary initial condition: 1/N-uniform vector of size N (initial condition does not matters)
z_0 = np.ones((N,1))/N

# set a tolerance to assess convergence to the limit
tol = 1e-5

# run the dynamics
z_old = z_0

while True:
    z_new = P.T @ z_old * (1-beta) + beta * mu
    if np.linalg.norm(z_new-z_old) < tol:
        break
    z_old=z_new

zb_distr = z_new

# normalize the centrality
zb_distr = zb_distr / sum(zb_distr)

print("Bonacich centrality: \n", zb_distr)

## Katz centrality

For Katz centrality, consider the following dynamics:

$$
\begin{cases} 
z(t+1) = \frac{(1-\beta)}{\lambda_W}W'z(t) + \beta \mu  \\
z(0) = z_0.
\end{cases}
$$

It is easy to prove that the dynamics converges to the Katz centrality.

**Remark**: notice that the proposed method is iterative (and therefore more efficient than direct method), but it is not distributed. Indeed, to perform the computation, every node needs to know $\lambda_W$, which is a global information on the network.

## Exercise
Compute the Katz centrality of the Zachary's karate club graph by iterative methods, and compare the results with direct methods.

**Hint**: use the definition of Katz centrality and same techniques as above

In [ ]:
# TO DO

# Testing sensitivity of measures
In this section we will check the dependence of centrality measures with respect to their paramenters and the sensitivity of the iterative algorithms to compute such measure with respect to the number of iterations.

## The effect of parameters
In our first experiment we analyze the dependence of Page Rank centrality on the parameter $\alpha=1-\beta$. We set distinct values for $\alpha$ while we fix the number of iterations, and run Page Rank. Then we plot the resulting Page Rank values with respect to $\alpha$.

In [ ]:
fig = plt.figure(2, figsize=(16,7))
ax1 = plt.subplot(121)
ax2 = plt.subplot(122)

pos = nx.spring_layout(G) 
nx.draw_networkx(G, pos, ax=ax1)

# we consider values for alpha from 0 to 1 with step size 0.25
alphas = np.arange(0, 1.25, 0.25)

for alp in alphas:
    # pagerank has parameters alpha and mu:
    # note that alpha = 1-beta and weight parameters mu are set to 1 by default
    pr = nx.pagerank(G, alpha=alp) 
    prval = list(pr.values())
    ax2.plot(prval, color=np.random.rand(3), label='alpha {0:.2f}'.format(alp))
    
ax2.legend()

plt.show()

Let's explain the previous result. Keep in mind that the parameter alpha used by `nx.pagerank` corresponds to $1-\beta$, and the centrality $z$ satisfies

$$
z = (1-\beta)P'z + \beta \mu
$$

- As $\alpha = 0$ ($\beta = 1$), $z = \mu$, i.e., Bonacich is equivalent to the intrinsic centrality $\mu$, the network does not play any role.

- As $\alpha = 1$ ($\beta = 0$), $z = P'z$, i.e., Bonacich is equivalent to invariant distribution centrality (the intrinsic centrality is irrelevant).

In between these two extreme cases there is a combination of network effects and intrinsic centrality.

## Exercise ##
Repeat the analysis. This time keep $\alpha$ fixed to 0.5 and select 3 different non-uniform vectors $\mu$ as `personalization` parameter to `pagerank`. How do you interpret the result?

## The effect of iteration number
In this section we consider a bigger network and we analyse the speed of convergence of iterative algorithms for computing centrality measures. 
 

Download the political blogs network (save it as a .gml file in the working directory of this notebook) and import it as a Graph object. We check the basic properties of $G$

In [ ]:
G = nx.read_gml('polblogs.gml')
print("Type of G:", type(G))

Since $G$ is a multidigraph, we define an equivalent digraph to compute the centralities (the function 'pagerank' does not work with multigraphs)

In [ ]:
GG = nx.DiGraph()
for n, nbrs in G.adjacency():
    # edict is a dictionary of dictionaries; 
    # the keys of edict are parallel edges from n to nbr;
    # the values of edict are dictionary,
    # containing attribute values of the corresponding edge
    for nbr, edict in nbrs.items(): 
        # each edge has weight=1, so total value is just  
        # the number of parallel edges
        total_value = len(edict) 
        GG.add_edge(n, nbr, weight = total_value)

The graph is very large, thus we cannot plot it.

In [ ]:
print(GG.number_of_nodes())

We now test the convergence speed of `nx.pagerank` algorithm:

In [ ]:
fig = plt.figure(1, figsize=(16,7))

# set 3 iteration numbers
iters = [10,15,50]
# define the position of the next plot in the subplot grid
position = 1
# create a list to collect the page rank values obtained in the three runs 
prvals = []

for max_iter in iters:
    # compute page rank
    pr = nx.pagerank(GG, max_iter = max_iter) 
    # compute page rank values
    prval = list(pr.values())
    # append the result to the list
    prvals.append(np.array(prval)) 
    # create a new sublot in the grid
    ax = fig.add_subplot(2,2,position)
    # plot the PR values
    ax.plot(prval, color=np.random.rand(3), label='{0:d} iterations'.format(max_iter))
    position+=1

# add a legend which contains all labels
# informations specified in previous plot calls
fig.legend()  
# we assume the values obtained with nx.pagerank()
# with no iterations constraints as a benchmark
benchmark = np.array(list(nx.pagerank(GG).values())) 
# we compute errors as norm of the differences wrt the benchmark
errors = [np.linalg.norm(prval-benchmark) for prval in prvals]
print("Errors:", errors)

plt.show()

`nx.pagerank` algorithm converges very fast, in 10 iterations! 

### Exercise
Check if our iterative algorithm for computing Bonacich centrality is as good as this by performing a similar analysis.

In [ ]:
# TO DO

## An interpretation of Katz (and Bonacich) centrality
Bonacich centrality (as well as Katz centrality) can be interpreted in terms of walks on the graphs.
We show this by an example.

We shall make use of an undirected graph and Katz centrality to simplify the analysis.

In [ ]:
G = nx.lollipop_graph(6,3)
G.add_edges_from([(9,8),(10,8),(11,8),(12,8),(13,8)])

pos = nx.spring_layout(G) 
nx.draw(G, pos, with_labels = True)

N = len(G)

# compute matrices of the graph
W = nx.adjacency_matrix(G)
W = W.toarray()
degrees = np.sum(W,axis=1)
D = np.diag(degrees)
P = np.linalg.inv(D) @ W

# compute the largest eigenvalue of W
w,v = np.linalg.eig(W)
w = w.real

lambda_max = max(w) 

plt.show()

We consider the iterative implementation of Katz centrality
$$
\begin{cases} 
z(t+1) = \frac{(1-\beta)}{\lambda_W}W'z(t) + \beta \mu  \\
z(0) = z_0.
\end{cases}
$$
with uniform $z_0$ and $\mu$.

In [ ]:
mu = np.ones((N,1))/N
beta = 0.15
# initial centrality distribution
z = np.ones((N,1))/N
z_reshape = z.reshape(N)

print("Centralities at iteration 0:", z_reshape)

Note that normalizing $\mu$ does not modify the centrality, since in

$$ 
z =  (\mathbf{I}-\frac{1-\beta}{\lambda_W} W')^{-1} \beta \mu 
$$

$\mu$ affects the normalization of $z$ only (same for Bonacich centrality). 

However, the normalization of $\mu$ affects the transient of the iteration to compute $z$. In particular, if one considers the **Bonacich centrality**, using $\mu, z_0$ such that $\mathbf{1}' \mu = \mathbf{1}' z_0 = \mathbf{1}$ is preferable, since it guarantees that, if $\mathbf{1}' z(0)=1$, then $\mathbf{1}' z(t)=1$ for every $t$. Indeed, if $\mathbf{1}' z(t-1)= 1$, then 

$$
\mathbf{1}' z(t) = (1-\beta) \mathbf{1}' P' z(t-1) + \beta \mathbf{1}'\mu = (1-\beta) \mathbf{1}' z(t-1) + \beta = (1-\beta) + \beta = 1
$$

### Back to Katz centrality

After 1 iteration,
$$
z(1) = \frac{(1-\beta)}{\lambda_W}W'z(0) + \beta \mu,
$$

which means that the centrality of a node is the a combination of its intrinsic centrality, and the centrality of the neighbors. Since the intrinsic centrality is the same for every node, if we start with a uniform $z(0)$, $z(1)$ depends only on the degree of the node.

Similar observation can be made for the Bonacich centrality.

In [ ]:
# after 1 iteration
z = W.T @ z * (1-beta)/lambda_max + beta * mu

z_reshape = z.reshape(N)

nodesize=z_reshape*7000

# plot centrality at iteration 0
plt.figure(1, figsize=(10,7))
# we draw the graph with same node position "pos" defined above
nx.draw(G,pos,
         with_labels=True,
         nodelist=list(G.nodes()), 
         # node size is proportional to centrality value
         node_size = nodesize, 
         # node's color reflects centrality values (higher dc = darker color)
         node_color=z_reshape,
         font_size=8,
         # node's colors are on the red scale
         cmap=plt.cm.Reds) 

print("Centralities at iteration 1:", z_reshape)

plt.show()

As the number of iterations grows, $z(n)$ takes into account also nodes at greater distance.

At the equilibrium, the centrality $z^*$ can be interpreted in terms of walks as follows:

\begin{equation}
\begin{aligned}
z^*&=\lim_{n \to +\infty}z(n)=\sum_{n = 0}^{\infty} \left(\frac{(1-\beta)}{\lambda_W}\right)^n (W')^n \beta \mu \\
   &= \beta \mu + \frac{(1-\beta)}{\lambda_W} (W') \beta \mu + \left(\frac{(1-\beta)}{\lambda_W}\right)^2 (W')^2 \beta \mu + \cdots
\end{aligned}
\end{equation}

If one considers node $i$

$$
z^*_i = \beta \mu_i + \frac{(1-\beta)}{\lambda_W}\beta \sum_{j} (W')_{ij} \mu_j + \left(\frac{(1-\beta)}{\lambda_W}\right)^2 \beta \sum_{j} ((W')^2)_{ij} \mu_j + \cdots
$$

**Interpretation**. Since $((W')^n)_{ij}$ is the number of paths of length $n$ from $j$ to $i$, the centrality of node $i$ is the sum of:
- its intrinsic centrality $\mu_i$, plus 
- the intrinsic centrality of its in-neighbors, i.e., $\sum_j W_{ji} \mu_j$, plus 
- the intrinsic centrality of the nodes connected by paths of length 2, and so on... 

Longer paths have a decreasing weight due to the term $(1-\beta)^n$.

**Question**: which node do you expect to have a higher Katz centrality? Why?

In [ ]:
# set a tolerance to assess convergence to the limit
tol = 1e-5

z_0 = np.ones(N,)/N
mu = np.ones(N,)/N

# run the dynamics
z_old = z_0
print()
while True:
    z_new = W.T @ z_old * (1-beta)/lambda_max + beta * mu
    if np.linalg.norm(z_new-z_old) < tol:
        break
    z_old=z_new

zk = z_new

# normalize the centrality
zk = zk/sum(zk)
zk = zk.reshape(N)

print(zk)

plt.figure(1, figsize=(10,7))
# we draw the graph with same node position "pos" defined above
nx.draw(G,pos,
         with_labels=True,
         nodelist=list(G.nodes()), 
         # node size is proportional to centrality value
         node_size = zk*7000, 
         # node's color reflects centrality values (higher dc = darker color)
         node_color=zk,
         font_size=8,
         # node's colors are on the red scale
         cmap=plt.cm.Reds) 

**Question**: how do you expect to be modified the centralities when using Bonacich instead of Katz? Focus on node 8 vs node 5.

**Hint**: recall the definition of the two centralities, i.e.,

- Katz: $z =  \frac{1-\beta}{\lambda_W} W' z + \beta \mu$
- Bonacich: $x = (1-\beta)P' x + \beta \mu$ 

In [ ]:
zb_dict = nx.algorithms.link_analysis.pagerank_alg.pagerank(G)

# check if the centrality are normalized
zb = np.array(list(zb_dict.values()))

print(zk)

plt.figure(1, figsize=(10,7))
# we draw the graph with same node position "pos" defined above
nx.draw(G,pos,
         with_labels=True,
         nodelist=list(G.nodes()), 
         # node size is proportional to centrality value
         node_size = zb*7000, 
         # node's color reflects centrality values (higher dc = darker color)
         node_color=zb,
         font_size=8,
         # node's colors are on the red scale
         cmap=plt.cm.Reds) 

plt.show()

## Exercise

Compute all the centralities for the this graph, plot them, and comment the results.

**Hint**: use the code introduced in the lectures to compute the degree, eigenvector, Katz and Bonacich centralities. For the invariant distribution centrality use the function `np.linalg.eig()` to find the invariant distribution of $P$. Use the code introduced in the last lecture to plot the centralities.

In [ ]:
# TO DO

# Flows over networks
In this part of the lab we discuss how maximum admissible flows on a capacitated network are related with cuts in the network by the Max Flow - Min Cut theorem. 

Let us start with the definition of flows from a source 's' to a destination 't' (called terminal nodes).

**Definition**: a s-t flow is a distribution such that:
- for every non-terminal node the incoming flow equals the outcoming flow (mass conservation);
- the outcoming flow from $s$ equals the incoming flow to $t$ (this quantity is called **throughput**).

### Example
The blue edge labels indicate the flow along the edge.

In [ ]:
G = nx.DiGraph()
G.add_edges_from([(0,1),(0,2),(1,2),(2,3),(1,3)])

pos = nx.spring_layout(G)

nx.draw_networkx_nodes(G, pos, node_color ='lightblue')

nx.draw_networkx_labels(G, pos)

nx.draw_networkx_edges(G, pos, arrows=True, width=1)

nx.draw_networkx_edge_labels(G, pos,edge_labels={(0,1):'4',(0,2):'1',(1,2):'2',(1,3):'2',(2,3):'3'},font_color='blue')

plt.show()

Observe that the flow distribution labelled in blue is a flow from 0 to 3. Indeed,
- for the non-terminal nodes 1 and 2, the incoming flow equals the outcoming flow;
- the total flow outcoming from 0 equals the total flow incoming to 3 (the throughput is 5).

## Max Flow - Min Cut Theorem
It is given a graph $G = (V, E)$ which represents a flow network and two vertices source $s$ and sink $t$ in it. Every edge $(u,v)$ has a capacity $c(u,v)$. We want to find the maximum possible flow from s to t with the following constraint:

1. Flow on an edge doesn’t exceed the given capacity of the edge.

In [ ]:
nx.draw_networkx_nodes(G, pos, node_color ='lightblue')

nx.draw_networkx_labels(G, pos)

nx.draw_networkx_edges(G, pos, arrows=True, width=1)

nx.draw_networkx_edge_labels(G, pos,edge_labels={(0,1):'2',(0,2):'1',(1,2):'4',(1,3):'3',(2,3):'2'},font_color='red')

plt.show()

**Question**: consider the graph above with terminal nodes 0 and 3, where the red labels denote the capacity of the edge, i.e., the maximal flow that can be sent along the edge. What is the maximal flow that can be sent from 0 to 3?

**Question**: give an intuitive answer on why a flow of throughput 4 cannot be sent.

**Definition (cut of a network)**: a s-t cut of the network $G=(N,E)$ is a partition of the nodes $\{U,U^C\}$, such that $s \in U$ and $t \in U_C$.

We shall see that the notion of cut is strongly related to the maximal flow that can be sent in the network. The cuts of the network above are:
- $U=\{0,1,2\},U^C=\{3\}$
- $U=\{0,1\},U^C=\{2,3\}$
- $U=\{0,2\},U^C=\{1,3\}$
- $U=\{0\},U^C=\{1,2,3\}$

**Definition (cut capacity)**: the capacity of a cut $\{U,U^C\}$ is
$$
C_{U} = \sum_{i \in U}\sum_{j \in U^C} c(i,j)
$$

**Max flow min cut Theorem**: the maximal flow that can be sent from $s$ to $t$ equals the minimal cut capacity among the s-t cuts of the network. 

**Back to the question**:
- $U=\{0,1,2\},U^C=\{3\}$ -> $C_U = 5$
- $U=\{0,1\},U^C=\{2,3\}$ -> $C_U = 8$
- $U=\{0,2\},U^C=\{1,3\}$ -> $C_U = 4$
- $U=\{0\},U^C=\{1,2,3\}$ -> $C_U = 3$

**Answer**: the 0-3 min-cut has capacity 3, thus the maximal flow from 0 to 3 is 3.

NetworkX has many functions useful for flow applications, e.g., `networkx.algorithms.flow.maximum_flow` and `networkx.algorithms.flow.minimum_cut`, which compute the maximum throughput and the value and the node partition of a minimum cut, respectively.

In [ ]:
# to find the maximal flow, the edges have to be labelled with a 'capacity' label
# we then modify the graph in such a way to include this information 
# (check the `networkx.algorithms.flow.maximum_flow` documentation for more info)

G[0][1]['capacity'] = 2
G[0][2]['capacity'] = 1
G[1][2]['capacity'] = 4
G[1][3]['capacity'] = 3
G[2][3]['capacity'] = 2

nx.algorithms.flow.maximum_flow(G,0,3)

# maximum_flow returns the maximal throughput, plus a dictionary containing the value of the flow that goes through each edge.

In [ ]:
nx.algorithms.flow.minimum_cut(G,0,3)

## Intervention in capacitated networks: two dual problems

### Adversarial intervention
**Question**: suppose you are an adversarial agent that aims at minimizing the flow that can be send from 0 to 3 by removing capacity subject to a budget constraint (or even disconnect the network). 
- Where do you remove the capacity? 
- What is the minimal capacity that you need to remove in order to disconnect the network?

**Answer** 
- Since the bottleneck of the flow is the min-cut of the network, the adversary should reduce the capacity of such a  cut. Thus, in this case, the adversary should reduce capacity of edges between the sets $\{0\}$ and the set $\{1,2,3\}$, which are $(0,1)$ and $(0,2)$. 
- The minimal capacity that needs to be removed equals the capacity of the min-cut of the network, which is 3.

While for this network the answer could be obvious, when the network is large the algorithms `networkx.algorithms.flow.maximum_flow` and `networkx.algorithms.flow.minimum_cut` are very useful.

In [ ]:
nx.draw_networkx_nodes(G, pos, node_color ='lightblue')

nx.draw_networkx_labels(G, pos)

nx.draw_networkx_edges(G, pos, arrows=True, width=1)

nx.draw_networkx_edge_labels(G,pos,edge_labels={(0,1):'2',
(0,2):'1',(1,2):'4',(1,3):'3',
(2,3):'2'},font_color='red')

plt.show()

**Cuts**
- $U=\{0,1,2\},U^C=\{3\}$ -> $C_U = 5$
- $U=\{0,1\},U^C=\{2,3\}$ -> $C_U = 8$
- $U=\{0,2\},U^C=\{1,3\}$ -> $C_U = 4$
- $U=\{0\},U^C=\{1,2,3\}$ -> $C_U = 3$

## Planner intervention
**Question**: Suppose you are a planner that aims at maximing the flow that can be send from 0 to 3. Where do you allocate the capacity?

**Answer**: you should allocate the capacity in such a way that the capacity of the mincut is maximized.

### Example 1

In [ ]:
G.remove_edge(0,2)

nx.draw_networkx_nodes(G, pos, node_color ='lightblue')

nx.draw_networkx_labels(G, pos)

nx.draw_networkx_edges(G, pos, arrows=True, width=1)

nx.draw_networkx_edge_labels(G,pos,edge_labels={(0,1):'2',(1,2):'4',(1,3):'1',
(2,3):'3'},font_color='red')

plt.show()

**Problem**: suppose you can allocate a capacity 4 (that can be distributed on several links). Where do you allocate it?

**Solution**: first compute the capacity of the cuts:
- $U=\{0,1,2\},U^C=\{3\}$ -> $C_U = 4$
- $U=\{0,1\},U^C=\{2,3\}$ -> $C_U = 5$
- $U=\{0,2\},U^C=\{1,3\}$ -> $C_U = 5$
- $U=\{0\},U^C=\{1,2,3\}$ -> $C_U = 2$

The min-cut is $\{0\},\{1,2,3\}$, thus the capacity should be allocated on the links between these set, i.e., on $(0,1)$.

Is allocating capacity 4 on $(0,1)$ optimal? NO!

In such a case, the cut capacities become:

- $U=\{0,1,2\},U^C=\{3\}$ -> $C_U = 4$
- $U=\{0,1\},U^C=\{2,3\}$ -> $C_U = 5$
- $U=\{0,2\},U^C=\{1,3\}$ -> $C_U = 5$
- $U=\{0\},U^C=\{1,2,3\}$ -> $C_U = 6$

and the corresponding throughput is 4.

Note instead that after allocating a capacity 2 on edge $(0,1)$, the cut capacities become

- $U=\{0,1,2\},U^C=\{3\}$ -> $C_U = 4$
- $U=\{0,1\},U^C=\{2,3\}$ -> $C_U = 5$
- $U=\{0,2\},U^C=\{1,3\}$ -> $C_U = 5$
- $U=\{0\},U^C=\{1,2,3\}$ -> $C_U = 4$,

with still a capacity 2 to be allocated.

In [ ]:
nx.draw_networkx_nodes(G, pos, node_color ='lightblue')

nx.draw_networkx_labels(G, pos)

nx.draw_networkx_edges(G, pos, arrows=True, width=1)

nx.draw_networkx_edge_labels(G,pos,edge_labels={(0,1):'4',(1,2):'4',(1,3):'1',
(2,3):'3'},font_color='red')

plt.show()

This remaining capacity should be allocated on both the first and the last cut, so the final solution is to allocate capacity 3 on the edge $(0,1)$, and 1 on the edge $(1,3)$ or $(2,3)$, which leads to cuts

- $U=\{0,1,2\},U^C=\{3\}$ -> $C_U = 5$
- $U=\{0,1\},U^C=\{2,3\}$ -> $C_U = 5$
- $U=\{0,2\},U^C=\{1,3\}$ -> $C_U = 5$
- $U=\{0\},U^C=\{1,2,3\}$ -> $C_U = 5$,

and maximal throughput 5.

### Example 2
In some cases, it is possible that the capacity of two min-cuts can be increased by improving only one edge.
Consider the following example:

In [ ]:
G.add_edge(0,2)
G.remove_edge(1,3)

nx.draw_networkx_nodes(G, pos, node_color ='lightblue')

nx.draw_networkx_labels(G, pos)

nx.draw_networkx_edges(G, pos, arrows=True, width=1)

nx.draw_networkx_edge_labels(G,pos,edge_labels={(0,1):'1',(0,2):'1',(1,2):'1',
(2,3):'3'},font_color='red')

plt.show()

**Question**: what is the maximal throughput that can be sent from 0 to 3?

The answer could be given by using NetworkX functions, but we do not use it now.

**Answer**: we start by computing the cut capacities:

- $U=\{0,1,2\},U^C=\{3\}$ -> $C_U = 3$
- $U=\{0,1\},U^C=\{2,3\}$ -> $C_U = 2$
- $U=\{0,2\},U^C=\{1,3\}$ -> $C_U = 4$
- $U=\{0\},U^C=\{1,2,3\}$ -> $C_U = 2$.

The maximal throughput is thus 2.

**Question 2**: suppose you can allocate a capacity 1 on the network. Where do you allocate it?

**Answer**: there are two mincuts, whose capacity is 2. 
- For the mincut $U=\{0,1\},U^C=\{2,3\}$,
$$
C_U = c(0,2) + c(1,2)
$$ 
- For the mincut $U=\{0\},U^C=\{1,2,3\}$,
$$
C_U = c(0,1) + c(0,2)
$$

Note that both the capacities can be improved by adding capacity 1 to the edge $(0,2)$. By doing this, the capacities become

- $U=\{0,1,2\},U^C=\{3\}$ -> $C_U = 3$
- $U=\{0,1\},U^C=\{2,3\}$ -> $C_U = 3$
- $U=\{0,2\},U^C=\{1,3\}$ -> $C_U = 4$
- $U=\{0\},U^C=\{1,2,3\}$ -> $C_U = 3$,

and the maximal throughput become 3.

Obviously, these type of considerations can be implemented algorithmically.

## Max Flow - Min Cut Theorem and Ford Fulkerson algorithm

**Credits**: The code for Ford-Fulkerson Algorithm is based on material that can be found on Wikipedia.

It is given a graph $G = (V, E)$ which represents a flow network, and two vertices source $s$ and sink $t$ in it. Every edge $(u,v)$ has a capacity $c(u,v)$. We want to find the maximum possible flow from $s$ to $t$ with the following constraints.

The desired flow $f$ is constructed iteratively starting from a zero-flow vector: at each round of the algorithm $f$ is updated in such a way that the capacity and balance constraints are not violated.

To do this, at each round, given the current flow vector $f$, we define the **residual network** $G_{f}(V,E_{f})$ to be the network with capacity $c_{f}(u,v)=c(u,v)-f(u,v)$. A search on the residual graph $G_f$ is performed to find an **augmenting path** (i.e., a path on the residual network with positive capacity on every edge of the path). If an augmenting path is found, this means that the current flow $f$ on that path can be increased.

### Ford–Fulkerson Algorithm 
**Inputs**: a graph $G=(V,E)$, a vector of flow capacities $c = (c(u,v))_{(u,v) \in E}$, a source node $s$, and a sink node $t$

**Output**: a flow $f = (f(u,v))_{(u,v) \in E}$ from $s$ to $t$ of maximum throughput

1. $f(u,v)\leftarrow 0$ for all edges $(u,v)$
2. While there is a path $p$ from $s$ to $t$ in $G_{f}$, such that $c_{f}(u,v)=c(u,v)-f(u,v)>0$ for all edges $(u,v)\in p$:
 1. Find $c_{f}(p)=\min\{c_{f}(u,v):(u,v)\in p\}$
 2. For each edge $(u,v)\in p:$
      1. $f(u,v)\leftarrow f(u,v)+c_{f}(p)$ (Increase the flow along the path)
      2. $f(v,u)\leftarrow f(v,u)-c_{f}(p)$ (Impose the symmetry of the flow)

Ford-Fulkerson Algorithm is called a **method**, instead of **algorithm**, because it does not specify univocally how to select paths.

## Example: iterative Ford-Fulkerson by hand

In [ ]:
G = nx.DiGraph()
G.add_edges_from([(0,1),(0,2),(1,2),(1,3),(2,3)])

pos = {0: (40, 20), 1: (60, 35), 2: (60, 5), 3: (80, 20)}

labels = ['0/1000','0/1000','0/1','0/1000','0/1000']

zip_operator = zip(G.edges(), labels)
labels = dict(zip_operator)

nx.draw_networkx_nodes(G, pos, node_color ='lightblue')

nx.draw_networkx_labels(G, pos)

nx.draw_networkx_edges(G, pos, arrows=True, width=1)

nx.draw_networkx_edge_labels(G,pos,edge_labels=labels,font_color='red')

plt.show()

We aim at finding the maximal flow from 0 to 3.

The edge labels show the flow running along the edges and the capacity of the edges.

We start with flow 0 on every path.

0-1-2-3, 0-1-3 and 0-2-3 are augmenting paths

We arbitrarly choose to add flow on 0-1-2-3.

The capacity of 0-1-2-3 is the minimal capacity of the edges in the path, which is 1.

In [ ]:
labels = ['1/1000','0/1000','1/1','0/1000','1/1000']

colors = ['red','black','red','black','red']

zip_operator = zip(G.edges(), labels)
labels = dict(zip_operator)

nx.draw_networkx_nodes(G, pos, node_color ='lightblue')

nx.draw_networkx_labels(G, pos)

nx.draw_networkx_edges(G, pos, arrows=True, width=1)

nx.draw_networkx_edge_labels(G,pos,edge_labels=labels,font_color='red')

plt.show()

We update flows:
- $f(0,1) = 1$
- $f(1,0) = -1$ (symmetry)
- $f(2,3) = 1$
- $f(3,2) = -1$ (symmetry)
- $f(1,2) = 1$
- $f(2,1) = -1$ (symmetry),

We update the residual capacities $c_f(i,j) = c(i,j) - f(i,j)$ accordingly:
- $c_f(0,1) = 1000-1 = 999$
- $c_f(1,0) = 0-(-1) = 1$
- $c_f(2,3) = 1000-1 = 999$
- $c_f(3,2) = 0-(-1) = 1$
- $c_f(1,2) = 1-1 = 0$
- $c_f(2,1) = 0-(-1) = 1$.

Note that edges $(1,0)$ and $(2,1)$, which are not in the original network, have now positive residual capacity, meaning that we can allocate some flow on them. Intuitively speaking, allocating some flow on $(2,1)$ means removing flow from $(1,2)$. Imposing the symmetry of the flow allows to remove flow after the allocation.

We now proceed to iteratively updating flows. There are three augmenting paths: 0-1-3, 0-2-3, 0-2-1-3.

We arbitrarly choose to allocate flow on the path 0-2-1-3.

The maximum flow that can be allocated is 1, because of the residual capacity constraint of edge  (2,1) .

In [ ]:
labels = ['1/1000','1/1000','0/1','1/1000','1/1000']

colors = ['black','red','red','red','black']

zip_operator = zip(G.edges(), labels)
labels = dict(zip_operator)

nx.draw_networkx_nodes(G, pos, node_color ='lightblue')

nx.draw_networkx_labels(G, pos)

nx.draw_networkx_edges(G, pos, arrows=True, width=1)

nx.draw_networkx_edge_labels(G,pos,edge_labels=labels,font_color='red')

plt.show()

After 1998 steps more...

In [ ]:
labels = ['1000/1000','1000/1000','0/1','1000/1000','1000/1000']

zip_operator = zip(G.edges(), labels)
labels = dict(zip_operator)

nx.draw_networkx_nodes(G, pos, node_color ='lightblue')

nx.draw_networkx_labels(G, pos)

nx.draw_networkx_edges(G, pos, arrows=True, width=1)

nx.draw_networkx_edge_labels(G,pos,edge_labels=labels,font_color='red')

plt.show()

There are no longer augmenting paths with residual capacity in the network.

Thus, the resulting flows correspond to the maximal flow that can be sent from 0 to 3.

The maximal throughput that can be sent from 0 to 3 is 2000. We can verify this by using NetworkX methods.

In [ ]:
# assign feature 'capacity' to every edge, and use networkX method for maximum flow

G[0][1]['capacity'] = 1000
G[0][2]['capacity'] = 1000
G[1][2]['capacity'] = 1
G[1][3]['capacity'] = 1000
G[2][3]['capacity'] = 1000

nx.algorithms.flow.maximum_flow(G,0,3)

As you have seen, the convergence of the Ford-Fulkerson algorithm may be slow. This depends crucially on the sequence of augmenting paths that we choose. However, the algorithm is guaranteed to converge in finite steps if the capacities are rational numbers.

We draw the initial graph.

In [ ]:
G = nx.DiGraph()
G.add_edges_from([(0,1),(0,2),(1,2),(1,3),(2,3)])

pos = {0: (40, 20), 1: (60, 35), 2: (60, 5), 3: (80, 20)}

labels = ['0/1000','0/1000','0/1','0/1000','0/1000']

zip_operator = zip(G.edges(), labels)
labels = dict(zip_operator)

nx.draw_networkx_nodes(G, pos, node_color ='lightblue')

nx.draw_networkx_labels(G, pos)

nx.draw_networkx_edges(G, pos, arrows=True, width=1)

nx.draw_networkx_edge_labels(G,pos,edge_labels=labels,font_color='red')

plt.show()

Let us use another sequence of augmenting paths.

We start allocating flow on the path 0-1-3, with residual capacity equal to 1000.

In [ ]:
labels = ['1000/1000','0/1000','0/1','1000/1000','0/1000']

colors = ['red','black','black','red','black']

zip_operator = zip(G.edges(), labels)
labels = dict(zip_operator)

nx.draw_networkx_nodes(G, pos, node_color ='lightblue')

nx.draw_networkx_labels(G, pos)

nx.draw_networkx_edges(G, pos, arrows=True, width=1)

nx.draw_networkx_edge_labels(G,pos,edge_labels=labels,font_color='red')

plt.show()

There is now only an augmenting path, which is 0-2-3, with residual capacity equal to 1000

In [ ]:
labels = ['1000/1000','1000/1000','0/1','1000/1000','1000/1000']

colors = ['black','red','black','black','red']

zip_operator = zip(G.edges(), labels)
labels = dict(zip_operator)

nx.draw_networkx_nodes(G, pos, node_color ='lightblue')

nx.draw_networkx_labels(G, pos)

nx.draw_networkx_edges(G, pos, arrows=True, width=1)

nx.draw_networkx_edge_labels(G,pos,edge_labels=labels,font_color='red')

plt.show()

There are not augmenting paths in the network, thus we have found the maximal flow in only 2 steps.

### Edmonds-Karp Algorithm
The Edmonds–Karp algorithm is an implementation of Ford–Fulkerson method, where the search order to find augmenting paths is defined. A shortest path that has available capacity is found by a breadth-first search.

In [ ]:
import collections

# This class represents a directed graph using adjacency matrix representation
class Graph:
    # __init__ is a reseved method in python classes.
    # In object oriented terminology, it is the constructor of the class.
    # We define a new class 'graph' instead of using the NetworkX package.
    def __init__(self, graph): 
        # the Graph object represents the flow network and the graph
        # attribute will be updated by the Edmonds-Karp Algorithm to 
        # represent the residual graph.
        # Entry [u][v] of graph array stores the capacity of link (u,v).
        self.graph = graph  
        # Number of nodes:
        self.ROW = len(graph)

    def bfs(self, s, t, parent):
        """Returns true if there is a path from source 's' to sink 't' in the
        residual graph. Also fills parent[] to store the path """
        # Mark all the vertices as not visited
        visited = [False] * (self.ROW)

        # Create a double-ended queue for BFS
        queue = collections.deque()

        # Mark the source node as visited and enqueue it
        # When visited, nodes are appended "to the right end" of the queue.
        queue.append(s)
        visited[s] = True

        # Standard BFS loop
        while queue: # while there are still elements in the queue
            # Nodes are extracted from "the left end" of the queue
            u = queue.popleft()

            # Get all adjacent vertices of the dequeued vertex u
            # If an adjacent vertex has not been visited, then mark 
            # it as visited and enqueue it.
            
            # Enumerate() method adds a counter to an iterable
            # to keep a count of iterations. We cycle over row
            # u of graph array: ind is the index of the node,
            # val is >0 if node ind is adjacent to u, i.e. if 
            # there is residual capacity on the link (u,ind),
            # it is 0 otherwise.
            for ind, val in enumerate(self.graph[u]): 
                if (visited[ind] == False) and (val > 0):
                    queue.append(ind)
                    visited[ind] = True
                    parent[ind] = u

        # If sink is reached by BFS starting from source, then return
        # true, else return false
        return visited[t]

    # Returns the maximum flow from s to t in the given graph
    def edmonds_karp(self, source, sink):
        # Both max flow value and max flow vector are 
        # initialized to 0
        max_flow = 0  
        # initialize matrix of zeros with size ROW x ROW
        flow = np.array([[0] * (self.ROW)] * (self.ROW))

        # parent array is filled by bfs() and it is used 
        # to compute augmenting paths.
        # It is initialized with the sentinel value -1
        parent = [-1] * (self.ROW)

        # Augment the flow while there is an augmenting path from source to sink
        # in the residual graph
        while self.bfs(source, sink, parent): # the algorithm stops when there are not augmenting paths from s to t

            # Find the minimum residual capacity of the edges along the
            # origin-destination path found by BFS.
            res_capacity = float("Inf")
            # start from the endpoint of the path
            n = sink
            # travel the path backwards until you reach the source
            # and update the minimal residual capacity of links
            while n != source:
                res_capacity = min(res_capacity, self.graph[parent[n]][n])
                n = parent[n] 

            # Add the residual capacity to the maximal throughput
            max_flow += res_capacity

            # Update the flow vector by adding the residual capacity
            # of the augmenting path to each edge in the path (to preserve
            # symmetry, subtract it to each reverse edge).
            # Compute the new residual network by updating
            # the residual capacities of the edges and reverse edges
            # along the augmenting path.
            v = sink
            while v != source:
                u = parent[v]
                flow[u][v] += res_capacity
                flow[v][u] -= res_capacity
                self.graph[u][v] -= res_capacity
                self.graph[v][u] += res_capacity
                v = parent[v]  
            
        return max_flow, flow

In [ ]:
W = np.array([[0,1000,1000,0],[0,0,1,1000],[0,0,0,1000],[0,0,0,0]])
G = Graph(W)

max_flow, flow = Graph.edmonds_karp(G,0,3)

print("The maximum flow:", max_flow, "\n")
print("The flow distribution: \n", flow)

Let us explore the steps of the algorithm by adding some print in the code.

In [ ]:
class Graph_print:
    # __init__ is a reseved method in python classes.
    # In object oriented terminology, it is the constructor of the class.
    # We define a new class 'graph' instead of using the NetworkX package.
    def __init__(self, graph): 
        # the Graph object represents the flow network and the graph
        # attribute will be updated by the Edmonds-Karp Algorithm to 
        # represent the residual graph.
        # Entry [u][v] of graph array stores the capacity of link (u,v).
        self.graph = graph  
        # Number of nodes:
        self.ROW = len(graph)

    def bfs(self, s, t, parent):
        """Returns true if there is a path from source 's' to sink 't' in the
        residual graph. Also fills parent[] to store the path """
        # Mark all the vertices as not visited
        visited = [False] * (self.ROW)

        # Create a double-ended queue for BFS
        queue = collections.deque()

        # Mark the source node as visited and enqueue it
        # When visited, nodes are appended "to the right end" of the queue.
        queue.append(s)
        visited[s] = True

        # Standard BFS loop
        while queue: # while there are still elements in the queue
            # Nodes are extracted from "the left end" of the queue
            print("Queue:", queue, "\n")
            u = queue.popleft()
            print("Extracted node:", u, "\n")

            # Get all adjacent vertices of the dequeued vertex u
            # If an adjacent vertex has not been visited, then mark 
            # it as visited and enqueue it.
            
            # Enumerate() method adds a counter to an iterable
            # to keep a count of iterations. We cycle over row
            # u of graph array: ind is the index of the node,
            # val is >0 if node ind is adjacent to u, i.e. if 
            # there is residual capacity on the link (u,ind),
            # it is 0 otherwise.
            for ind, val in enumerate(self.graph[u]): 
                if (visited[ind] == False) and (val > 0):
                    queue.append(ind)
                    visited[ind] = True
                    parent[ind] = u
            
            print("Parents", parent, "\n")
            print("Visited", visited, "\n \n \n")

        # If sink is reached by BFS starting from source, then return
        # true, else return false
        return visited[t]

    # Returns the maximum flow from s to t in the given graph
    def edmonds_karp(self, source, sink):
        n_iter = 1
        print("iteration of bfs number:", n_iter, "\n")
        # parent array is filled by bfs() and it is used 
        # to compute augmenting paths.
        # It is initialized with the sentinel value -1
        # If parent[j] == -1, node 'j' has not been visited yet
        parent = [-1] * (self.ROW)

        # Both max flow value and max flow vector are 
        # initialized to 0
        max_flow = 0  
        # initialize matrix of zeros with size ROW x ROW
        flow = np.array([[0] * (self.ROW)] * (self.ROW))

        # Augment the flow while there is a path from source to sink
        # in the residual graph
        while self.bfs(source, sink, parent): # the algorithm stops when there are not augmenting paths from s to t
            # now parent stores the augmenting path found by bfs

            # Find the minimum residual capacity of the edges along the
            # origin-destination path found by BFS.
            res_capacity = float("Inf")
            # start from the endpoint of the path
            n = sink
            # travel the path backwards until you reach the source
            # and update the minimal residual capacity of links
            while n != source:
                res_capacity = min(res_capacity, self.graph[parent[n]][n])
                n = parent[n] 

            # Add the residual capacity to the maximal throughput
            max_flow += res_capacity

            # Update the flow vector by adding the residual capacity
            # of the augmenting path to each edge in the path (to preserve
            # symmetry, subtract it to each reverse edge).
            # Compute the new residual network by updating
            # the residual capacities of the edges and reverse edges
            # along the augmenting path.
            v = sink
            while v != source:
                u = parent[v]
                flow[u][v] += res_capacity
                flow[v][u] -= res_capacity
                self.graph[u][v] -= res_capacity
                self.graph[v][u] += res_capacity
                v = parent[v]  
            n_iter += 1
            print("iteration of bfs number:", n_iter, "\n")
            
        return max_flow, flow

In [ ]:
W = np.array([[0,1000,1000,0],[0,0,1,1000],[0,0,0,1000],[0,0,0,0]])
G = Graph_print(W)

max_flow, flow = Graph_print.edmonds_karp(G,0,3)

Networkx implements the Edmonds-Karp algorithm. 

The `networkx.algorithms.flow.edmonds_karp` function returns the residual network resulting after computing the maximum flow.

Networkx also provides the functions `networkx.algorithms.flow.maximum_flow` and `networkx.algorithms.flow.minimum_cut` which compute the maximun throughput and  the value and the node partition of a minimum cut, respectively.

## Multi origin-destination maxflow

So far, we have studied maximum flow problems when there is a single origin-destination pair. 

For some applications, it is useful to find maximum flow with multiple origin-destinations.

The multi o-d maxflow problem may be rephrased in terms of single o-d maxflow problem, and then handled with the standard tools. Let us see this by an example.

In [ ]:
G = nx.DiGraph()
G.add_edges_from([(0,2),(0,1),(2,4),(1,3),(2,5),(3,4),(4,5),(1,4),(1,5)])

pos = {0: [0,2], 1: [1,0], 2: [1,3], 3: [3,0], 4: [3,2], 5:[1,2] }

nx.draw_networkx_nodes(G, pos, node_color ='lightblue')

nx.draw_networkx_labels(G, pos)

nx.draw_networkx_edges(G, pos, arrows=True, width=1)

nx.draw_networkx_edge_labels(G,pos,edge_labels={(0,1):'4',
(0,2):'6',(2,4):'2',(2,5):'5',
(3,4):'6', (4,5):'1', (1,3):'2', (1,4):'3', (1,5): ' 5'},font_color='blue')

plt.show()

Consider sources $\{0,1\}$ and sinks $\{4,5\}$. To reconduct the problem to a single source and sink, an equivalent problem is to add an additional pair of nodes $S$ and $T$, and add edges from $S$ to every source of the original network, and from every sink of the original network to $T$, with infinite capacity.

For simplicity, let us call 6 and 7 the new nodes.

We define the resulting network for this example.

In [ ]:
G1 = nx.DiGraph()
G1.add_edges_from([(0,2),(0,1),(2,4),(1,3),(2,5),(3,4),(4,5),(1,4),(1,5)])

# add additional nodes and edges
G1.add_edges_from([(6,0),(6,1),(4,7),(5,7)])

# define position for the two new nodes
pos = {0: [0,2], 1: [1,0], 2: [1,3], 3: [3,0], 4: [3,2], 5:[1,2], 6:[-1,1], 7:[2,0.5]}

nx.draw_networkx_nodes(G1, pos, node_color ='lightblue')

nx.draw_networkx_labels(G1, pos)

nx.draw_networkx_edges(G1, pos, arrows=True, width=1)

nx.draw_networkx_edge_labels(G,pos,edge_labels={(0,1):'4',
(0,2):'6',(2,4):'2',(2,5):'5',
(3,4):'6', (4,5):'1', (1,3):'2', (1,4):'3', (1,5): ' 5',
(6,0):'inf',(6,1):'inf',(4,7):'inf',(5,7):'inf'},font_color='blue')

plt.show()

**Proposition**: the max-flow in the new network with source S and sink T is equivalent to the max-flow of the original problem.

In [ ]:
# We define the capacities, and use NetworkX algorithms to compute maximum flow

G1[0][2]['capacity'] = 6
G1[0][1]['capacity'] = 4
G1[2][4]['capacity'] = 4
G1[2][5]['capacity'] = 5
G1[3][4]['capacity'] = 6
G1[4][5]['capacity'] = 1
G1[1][3]['capacity'] = 2
G1[1][4]['capacity'] = 3
G1[1][5]['capacity'] = 5

# call S = 6, and T = 7, and assign infinite capacity
G1[6][0]['capacity'] = float('inf')
G1[6][1]['capacity'] = float('inf')
G1[4][7]['capacity'] = float('inf')
G1[5][7]['capacity'] = float('inf')

nx.algorithms.flow.maximum_flow(G1,6,7)

In [ ]:
# compute also the minimum cut
nx.algorithms.flow.minimum_cut(G1,6,7)

As expected, the minimum cut equals the maximal flow.

## Application of multi o-d maxflow problems: perfect matching

**Definition**: given a graph $G=(V,E)$, a matching is a subset of edges $M \subseteq E$ such that no edges in $M$ share a common node (head or tail).

**Definition**: Consider a simple bipartite graph, whereby the node set can be partitioned as $V = V_0 \cup V_1$, with $V_0 \cap V_1 = \emptyset$, and no edges between nodes of the same set exist. For $h = 0,1$, we shall refer to a matching $M$ in $G$ as $V_h$-perfect (or $V_h$-saturating) if every node in $V_h$ is matched in $M$.

### Example

In [ ]:
G = nx.Graph()
G.add_edges_from([(0,2),(0,3),(1,3),(1,4)])

pos = {0:[0,2], 1:[0,0], 2:[2,3], 3:[2,1], 4:[2,-1]}

nx.draw(G, pos, with_labels=True)

plt.show()

We immediately observe that the graph is bipartite, with $V_0 = \{0,1\}, V_1 = \{2,3,4\}$.

We now provide a $V_0$-perfect matching, i.e., a matching that matches every node in $V_0$

In [ ]:
G = nx.Graph()
G.add_edges_from([(0,2),(1,3)])
G.add_node(4)

pos = {0:[0,2], 1:[0,0], 2:[2,3], 3:[2,1], 4:[2,-1]}

nx.draw(G, pos, with_labels=True)

plt.show()

**Question**: can you find a $V_1$ perfect matching?

**Hall's marriage Theorem**: for a simple bipartite graph $G = (V,E)$ and $V_0 \subseteq V$, there exists a $V_0$-perfect matching in $G$ if and only if

$$
|\mathcal{U}| \le |\mathcal{N}_{\mathcal{U}}| \quad \forall \mathcal{U} \subseteq V_0,
$$

where $\mathcal{N}_{\mathcal{U}}$ is the neighborhood of $\mathcal{U}$ in $G$.

In other words, for every subset $U$ of $V_0$, we require that $U$ possesses a number of neighboring nodes which is at least the number of nodes in $U$. The necessity of such condition is trivial to verify, for the sufficiency we refer to the lecture notes.

We here focus on the algorithmic viewpoint of matching problems and on applications.

**Example**: assume there are 5 people and 5 books (with one copy of each book), and each person expresses interest in some of the books. Is there a way to assign to every person a book of interest?

We can represent this situation by a simple bipartite network, where $V_0$ is the set of people and $V_1$ and the set of books, and an edge $(n_0,n_1)$ (with $n_0 \in V_0$, $n_1 \in V_1$) exists if the person $n_0$ is interested in the book $n_1$. 

The question may be rephrased in: "does a $V_0$-perfect matching exist?" 

If there are less books than people, i.e., if $|V_1|<|V_0|$, a $V_0$-perfect matching cannot exist. This follows immediately from Hall's theorem by taking $\mathcal{U}=V_0$, and observing that $N_{\mathcal{U}} \subseteq V_1$.

**Remark**: Note that a perfect matching (both for $V_0$ and $V_1$) can exist only if $|V_0|=|V_1|$.

We consider a simple bipartite network, and try a naive algorithm to answer the question.

In [ ]:
G = nx.Graph()
G.add_edges_from([(0,6),(0,7),(1,6),(1,7),(1,8),(2,5),(2,6),(2,7),(2,9),(3,7),(4,7),(4,8),(4,9)])

pos = {0:[0,2], 1:[0,1], 2:[0,0], 3:[0,-1], 4:[0,-2], 5:[1,2], 6:[1,1], 7:[1,0], 8:[1,-1], 9:[1,-2]}

nx.draw(G,pos, with_labels=True)

plt.show()

We let the left nodes be people, and the right nodes be books. You can verify that the graph is bipartite.

We try this greedy algorithm: for every person (from above to below) we assign the first book (from above to below) of interest.

In [ ]:
# Assign book 6 to person 0
for u,v in G.edges():
    G[u][v]['color'] = 'black'
    
G[0][6]['color'] = 'red'

colors = [G[u][v]['color'] for u,v in G.edges()]

nx.draw(G, pos, with_labels = True, edge_color = colors)

plt.show()

In [ ]:
# Book 6 is not available for person 1, thus assign book 7
G[1][7]['color'] = 'blue'

colors = [G[u][v]['color'] for u,v in G.edges()]

nx.draw(G, pos, with_labels = True, edge_color = colors)

plt.show()

In [ ]:
# Assign book 5 to person 2
G[2][5]['color'] = 'green'

colors = [G[u][v]['color'] for u,v in G.edges()]

nx.draw(G, pos, with_labels = True, edge_color = colors)

plt.show()

Node 3 is not matched! The greedy algorithm fails in finding a perfect matching. 

However, this does not mean that a perfect matching does not exist. To verify that a perfect matching exists, one could verify the sufficient and necessary condition in Hall's marriage theorem. 

We instead show an analogy between maximal flows and perfect matching, and exploit such an analogy to find perfect matching by Ford-Fulkerson theorem.

Given a simple bipartite graph $G=(V,E)$, consider the directed capacitated graph $G1$, with node set $V \cup s \cup t$, and edge set constructed as follows:
- for every node $n \in V_0$, add an edge (s,n), with capacity 1;
- for every node $n \in V_1$, add an edge (n,t), with capacity 1;
- for every undirected edge $(i,j)$ in $G$, add a directed edge $(i,j)$ in $G1$ with capacity 1.

Let us construct $G1$.

In [ ]:
G1 = nx.DiGraph()
G1.add_edges_from([(0,6),(0,7),(1,6),(1,7),(1,8),(2,5),(2,6),(2,7),(2,9),(3,7),(4,7),(4,8),(4,9)])
G1.add_edges_from([(10,0),(10,1),(10,2),(10,3),(10,4),(5,11),(6,11),(7,11),(8,11),(9,11)])

pos = {0:[0,2], 1:[0,1], 2:[0,0], 3:[0,-1], 4:[0,-2], 5:[1,2], 6:[1,1], 7:[1,0], 8:[1,-1], 9:[1,-2], 10:[-1,0], 11:[2,0]}

nx.draw(G1,pos,with_labels=True)

plt.show()

There is an analogy between perfect matchings and maximal flow on this auxiliary network $G1$. In particular, a $V_0$-perfect matching on $G$ exists if and only if it there exists a flow with throughput $|V_0|$ on the network $G1$. We can now exploit Ford-Fulkerson algorithm to find the maximum flow that can be sent in $G1$.

**Remark**: the analogy can be proved by using the fact that, if the capacities of the edges are integers, then the maximum flow has integer components (see the lecture notes). Because of the network structure, if a maximum flow with throughput $|V_0|$ exists, then every node in $V_0$ has incoming and outcoming flow $1$, since all the edges have unitary capacity. Moreover, given a node $i \in |V_0|$, if a $|V_0|$-perfect matching exists, then there exists $k \in V_1$ such that

$$
f_{(i,j)} = 
\begin{cases}
1 \quad \text{if} \ j = k \\
0 \quad \text{otherwise}
\end{cases}
$$

Thus, given a maximum flow with throughput $|V_0|$, the associated $V_0$-perfect matching can be found by selecting all the edges $(i,j)$ ($i \in V_0, j \in V_1$) such that $f_{(i,j)}=1$.

**Exercise**: find the maximum flow on $G1$, and deduce the perfect matching on $G$.

### Exercise Katz centrality

In [ ]:
W = nx.adjacency_matrix(G)
W = W.toarray()

# compute the largest eigenvalue of W
w,v = np.linalg.eig(W)
w = w.real

lambda_max = max(w) 

beta = 0.15
mu = np.ones((N,1))

z_0 = np.ones((N,1))/N
# set a tolerance to assess convergence to the limit
tol = 1e-5
# evolve the dynamics
z_old = z_0

while True:
    z_new = W.T @ z_old * (1-beta)/lambda_max + beta * mu
    if np.linalg.norm(z_new-z_old) < tol:
        break
    z_old=z_new

zk_approx = z_new

# normalize the centrality
zk_approx = zk_approx / sum(zk_approx)

### Exercise speed of convergence Bonacich

In [ ]:
fig = plt.figure(1, figsize=(16,7))

# set 3 iteration numbers
iters = [10,15,50]
# define the position of the next plot in the subplot grid
position = 1
# create a list to collect the page rank values obtained in the three runs 
prvals = []

for max_iter in iters:
    # compute page rank
    z_0 = np.ones((N,1))/N
    z_old = z_0
    
    while True:
        z_new = P.T @ z_old * (1-beta) + beta * mu
        if np.linalg.norm(z_new-z_old) < tol:
            break
        z_old=z_new

    pr = z_new

    # normalize the centrality
    prval = pr / sum(pr)
    
    # append the result to the list
    prvals.append(np.array(prval)) 
    # create a new sublot in the grid
    ax = fig.add_subplot(2,2,position)
    # plot the PR values
    ax.plot(prval, color=np.random.rand(3), label='{0:d} iterations'.format(max_iter))
    position+=1

# add a legend which contains all label
# informations specified in previous plot calls
fig.legend()